## Simple Pendulum

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
me.init_vprinting()

In [ ]:
theta, omega = me.dynamicsymbols("theta, omega")
thetad = me.dynamicsymbols("theta", 1)
L = symbols("L")

M, N, A = symbols("M, N, A", cls=me.ReferenceFrame)
N.orient_axis(M, M.z, pi)
A.orient_axis(N, N.z, theta)

O, P, G = symbols("O, P, G", cls=me.Point)
G.set_pos(O, L / 2 * A.y)
P.set_pos(O, L * A.y)

O.set_vel(N, 0)
G.v2pt_theory(O, N, A)
P.v2pt_theory(O, N, A)

In [ ]:
q_ind = [theta]
u_ind = [omega]
kde = [thetad - omega]

kane = me.KanesMethod(
    N,
    q_ind=q_ind,
    u_ind=u_ind,
    kd_eqs=kde
)

In [ ]:
m, rho, g = symbols("m, rho, g")
cord_mass = rho * L
cord = me.RigidBody(
    "Cord", G, A, cord_mass,
    (me.inertia(A, 0, 0, rho * L**3 / 12), G))
particle = me.Particle("Particle", P, m)

In [ ]:
bodies = [cord, particle]
loads = [
    (G, cord_mass * g * N.y),
    (P, m * g * N.y)
]

In [ ]:
fr, frstar = kane.kanes_equations(bodies, loads)
fr + frstar

In [ ]:
kane.kindiffdict??

In [ ]:
thetad_val = kane.kindiffdict()
thetad_val

In [ ]:
eom = kane.rhs().subs(thetad_val)
eom

In [ ]:
eom[1].subs(rho, 0)

In [ ]:
t = symbols("t")
lhs = theta.diff(t, 2)
rhs = eom[1].subs(rho, 0)
Eq(lhs, rhs)

### Extraction of information.

In [ ]:
mm = kane.mass_matrix_full
fm = kane.forcing_full
mm.inv() * fm

In [ ]:
import numpy as np
from pydy.system import System

sys = System(kane)
sys.constants = {
    m: 1,
    rho: 0,
    L: 2,
    g: 9.81
}
sys.initial_conditions = {
    theta: np.pi/6,
    omega: 0
}

fps = 60
t0, tf = 0, 5
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)

results = sys.integrate()

In [ ]:
sys.integrate??

In [ ]:
import matplotlib.pyplot as plt

results_deg = np.rad2deg(results)
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(sys.times, results_deg[:, 0])
ax[0].set_ylabel(r"$\theta$ [deg]")
ax[1].plot(sys.times, results_deg[:, 1])
ax[1].set_ylabel(r"$\omega$ [deg / s]")
ax[1].set_xlabel("Time [s]")
plt.show()

### Animation with PyDy

In [ ]:
from pydy.viz import Scene, Cylinder, Sphere, VisualizationFrame

In [ ]:
VisualizationFrame??

In [ ]:
cord_geom = Cylinder(length=L, radius=0.05, color='grey', name="cord")
particle_geom = Sphere(0.2, color="black", name="particle")

cord_vf = VisualizationFrame('cord vf', A, G, cord_geom)
particle_vf = VisualizationFrame('particle vf', N, P, particle_geom)

# arguments to Scene: reference frame, origin, visual frames, system
scene = Scene(M, O, cord_vf, particle_vf, system=sys)
scene.display_jupyter(axes_arrow_length=1)

In [ ]:
phi = 0

# new reference frame for the cord
AA = me.ReferenceFrame("A")
# rotate the cord about NN.z by some value
AA.orient_axis(M, M.z, phi)

OO = me.Point("OO")
PP1 = OO.locatenew("PP1", L * AA.x)
GG1 = OO.locatenew("GG1", L / 2 * AA.x)
PP2 = OO.locatenew("PP2", L * AA.y)
GG2 = OO.locatenew("GG2", L / 2 * AA.y)

cylinder_x = Cylinder(
    length=L, radius=0.1, name="cylinder aligned with x", color="red")
cylinder_x_vf = VisualizationFrame('cylinder x vf', AA, GG1, cylinder_x)

cylinder_y = Cylinder(
    length=L, radius=0.1, name="cylinder aligned with y", color="green")
cylinder_y_vf = VisualizationFrame('cylinder y vf', AA, GG2, cylinder_y)

scene = Scene(M, OO, cylinder_x_vf, cylinder_y_vf, system=sys)
scene.display_jupyter(axes_arrow_length=0.5)

In [ ]:
phi = pi / 6

# new reference frame for the cord
AA = me.ReferenceFrame("A")
# rotate the cord about NN.z by some value
AA.orient_axis(M, M.z, phi)

OO = me.Point("OO")
PP1 = OO.locatenew("PP1", L * AA.x)
GG1 = OO.locatenew("GG1", L / 2 * AA.x)
PP2 = OO.locatenew("PP2", L * AA.y)
GG2 = OO.locatenew("GG2", L / 2 * AA.y)

cylinder_x = Cylinder(
    length=L, radius=0.1, name="cylinder aligned with x", color="red")
cylinder_x_frame = me.ReferenceFrame("cf")
cylinder_x_frame.orient_axis(AA, AA.z, -pi/2)
cylinder_x_vf = VisualizationFrame('cylinder x vf', cylinder_x_frame, GG1, cylinder_x)

cylinder_y = Cylinder(
    length=L, radius=0.1, name="cylinder aligned with y", color="green")
cylinder_y_vf = VisualizationFrame('cylinder y vf', AA, GG2, cylinder_y)

scene = Scene(M, OO, cylinder_x_vf, cylinder_y_vf, system=sys)
scene.display_jupyter(axes_arrow_length=0.5)

### Animation with Matplotlib

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots()
xp = sys.constants[L] * np.sin(results[0, 0])
yp = -sys.constants[L] * np.cos(results[0, 0])
string_handle, = ax.plot([0, xp], [0, yp], color="darkgray")
mass_handle = ax.scatter([xp], [yp], color="k")
ax.set_aspect("equal")
ax.axis([-2, 2, -2.5, 0.5])

def update(idx):
    xp = sys.constants[L] * np.sin(results[idx, 0])
    yp = -sys.constants[L] * np.cos(results[idx, 0])
    string_handle.set_data([0, xp], [0, yp])
    mass_handle.set_offsets([xp, yp])
    ax.set_title("Time = {:.2f} s".format(sys.times[idx]))

ani = FuncAnimation(fig, update, frames=len(sys.times))
HTML(ani.to_jshtml(fps=fps))
# optionally, save it to disk
# ani.save("simple-pendulum.mp4", fps=fps)

### Adding Friction to the Pendulum

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
from pydy.system import System
import numpy as np
from pydy.viz import Scene, Cylinder, Sphere, VisualizationFrame

me.init_vprinting()

theta, omega = me.dynamicsymbols("theta, omega")
thetad = me.dynamicsymbols("theta", 1)
L = symbols("L")

M, N, A = symbols("M, N, A", cls=me.ReferenceFrame)
N.orient_axis(M, M.z, pi)
A.orient_axis(N, N.z, theta)

O, P, G = symbols("O, P, G", cls=me.Point)
G.set_pos(O, L / 2 * A.y)
P.set_pos(O, L * A.y)

O.set_vel(N, 0)
G.v2pt_theory(O, N, A)
P.v2pt_theory(O, N, A)

q_ind = [theta]
u_ind = [omega]
kde = [thetad - omega]

kane = me.KanesMethod(
    N,
    q_ind=q_ind,
    u_ind=u_ind,
    kd_eqs=kde
)

m, rho, g = symbols("m, rho, g")
cord_mass = rho * L
cord = me.RigidBody(
    "Cord", G, A, cord_mass,
    (me.inertia(A, 0, 0, rho * L**3 / 12), G))
particle = me.Particle("Particle", P, m)

bodies = [cord, particle]
d = symbols("d")
loads = [
    (G, cord_mass * g * N.y),
    (P, m * g * N.y),
    (A, -d * omega * A.z),
    (N, d * omega * A.z),
]

fr, frstar = kane.kanes_equations(bodies, loads)
display(fr + frstar)

sys = System(kane)
sys.constants = {
    m: 1,
    rho: 0,
    L: 2,
    g: 9.81,
    d: 1.25
}
sys.initial_conditions = {
    theta: np.pi/6,
    omega: 0
}

fps = 60
t0, tf = 0, 10
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)

results = sys.integrate()

import matplotlib.pyplot as plt

results_deg = np.rad2deg(results)
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(sys.times, results_deg[:, 0])
ax[0].set_ylabel(r"$\theta$ [deg]")
ax[1].plot(sys.times, results_deg[:, 1])
ax[1].set_ylabel(r"$\omega$ [deg / s]")
ax[1].set_xlabel("Time [s]")
plt.show()

cord_geom = Cylinder(length=L, radius=0.05, color='grey', name="cord")
particle_geom = Sphere(0.2, color="black", name="particle")

cord_vf = VisualizationFrame('cord vf', A, G, cord_geom)
particle_vf = VisualizationFrame('particle vf', N, P, particle_geom)

# arguments to Scene: reference frame, origin, visual frames, system
scene = Scene(M, O, cord_vf, particle_vf, system=sys)
scene.display_jupyter(axes_arrow_length=1)

## Simple Pendulum wit Joints Framework

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
me.init_vprinting()

In [ ]:
theta, omega = me.dynamicsymbols("theta, omega")
m, rho, g, L = symbols("m, rho, g, L")

In [ ]:
M, N, A, B = symbols("M, N, A, B", cls=me.ReferenceFrame)
N.orient_axis(M, M.z, pi)
B.orient_axis(A, A.z, 0)

In [ ]:
wall = me.Body("Wall", frame=N)
# inertia wrt the cord's center of mass
cord_inertia = me.inertia(A, 0, 0, rho * L**3 / 12)
cord = me.Body("Cord", frame=A, mass=rho * L, central_inertia=cord_inertia)
particle = me.Body("Particle", mass=m)

In [ ]:
print(type(cord), type(particle))

In [ ]:
dir(cord)

In [ ]:
cord.apply_force??

In [ ]:
cord.apply_force(rho * L * g * N.y)
particle.apply_force(m * g * N.y)

In [ ]:
joint1 = me.PinJoint(
    "J1", wall, cord, coordinates=theta, speeds=omega,
    child_point=-L / 2 * cord.frame.y,
    joint_axis=wall.frame.z
)

In [ ]:
joint2 = me.WeldJoint(
    "J2", cord, particle,
    parent_point=L/2 * cord.frame.y,
    parent_interframe=B
)

In [ ]:
O = cord.masscenter.locatenew("O", -L/2 * cord.frame.y)
particle.masscenter.pos_from(O)

In [ ]:
tmp_cord_inertia = me.inertia(N, 0, 0, rho * L**3 / 12)
tmp_cord = me.Body("Cord", mass=rho * L, central_inertia=cord_inertia)
tmp_particle = me.Body("Particle", mass=m)
tmp_joint1 = me.PinJoint(
    "J1", wall, tmp_cord, coordinates=theta, speeds=omega,
    child_point=-L / 2 * tmp_cord.frame.y,
    joint_axis=wall.z)
tmp_joint2 = me.WeldJoint(
    "J2", tmp_cord, tmp_particle,
    parent_point=L/2 * tmp_cord.frame.y,
    parent_interframe=B)
tmp_O = tmp_cord.masscenter.locatenew("O", -L/2 * tmp_cord.frame.y)
tmp_particle.masscenter.pos_from(O)

In [ ]:
me.JointsMethod??

In [ ]:
method = me.JointsMethod(wall.frame, joint1, joint2)

kane = me.KanesMethod(
    wall.frame,
    q_ind=[theta],
    u_ind=[omega],
    kd_eqs=method.kdes,
    forcelist=method.loads,
    bodies=method.bodies
)

fr, frstar = kane.kanes_equations()
trigsimp(fr + frstar)

In [ ]:
import numpy as np
from pydy.system import System

sys = System(kane)
sys.constants = {
    m: 1,
    rho: 0,
    L: 2,
    g: 9.81
}
sys.initial_conditions = {
    theta: np.pi/6,#7*np.pi/4,
    omega: 0
}

fps = 60
t0, tf = 0, 5
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)

results = sys.integrate()

In [ ]:
from pydy.viz import Scene, Cylinder, Sphere, VisualizationFrame

cord_geom = Cylinder(length=L, radius=0.05, color='grey', name="cord")
particle_geom = Sphere(0.2, color="black", name="particle")

cord_vf = VisualizationFrame('cord vf', A, cord.masscenter, cord_geom)
particle_vf = VisualizationFrame('particle vf', N, particle.masscenter, particle_geom)

O = cord.masscenter.locatenew("O", -L/2 * cord.frame.y)
scene = Scene(M, O, cord_vf, particle_vf, system=sys)
scene.display_jupyter(axes_arrow_length=1)

### Adding friction to the Pendulum

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
me.init_vprinting()

theta, omega = me.dynamicsymbols("theta, omega")
m, rho, g, L, d = symbols("m, rho, g, L, d")

M, N, A, B = symbols("M, N, A, B", cls=me.ReferenceFrame)
N.orient_axis(M, M.z, pi)
B.orient_axis(A, A.z, 0)

wall = me.Body("Wall", frame=N)
# inertia wrt the cord's center of mass
cord_inertia = me.inertia(A, 0, 0, rho * L**3 / 12)
cord = me.Body("Cord", frame=A, mass=rho * L, central_inertia=cord_inertia)
particle = me.Body("Particle", mass=m)

cord.apply_force(rho * L * g * N.y)
cord.apply_torque(-d * omega * cord.frame.z, reaction_body=wall)
particle.apply_force(m * g * N.y)

joint1 = me.PinJoint(
    "J1", wall, cord, coordinates=theta, speeds=omega,
    child_point=-L / 2 * cord.frame.y,
    joint_axis=wall.z
)
joint2 = me.WeldJoint(
    "J2", cord, particle,
    parent_point=L/2 * cord.frame.y,
    parent_interframe=B
)

method = me.JointsMethod(wall.frame, joint1, joint2)

kane = me.KanesMethod(
    wall.frame,
    q_ind=[theta],
    u_ind=[omega],
    kd_eqs=method.kdes,
    forcelist=method.loads,
    bodies=method.bodies
)

fr, frstar = kane.kanes_equations()

import numpy as np
from pydy.system import System

sys = System(kane)
sys.constants = {
    m: 1,
    rho: 0,
    L: 2,
    g: 9.81,
    d: 1.25
}
sys.initial_conditions = {
    theta: np.pi/6,
    omega: 0
}

fps = 60
t0, tf = 0, 10
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)

results = sys.integrate()

display(kane.rhs()[1].subs(rho, 0))

import matplotlib.pyplot as plt

results_deg = np.rad2deg(results)
fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(sys.times, results_deg[:, 0])
ax[0].set_ylabel(r"$\theta$ [deg]")
ax[1].plot(sys.times, results_deg[:, 1])
ax[1].set_ylabel(r"$\omega$ [deg / s]")
ax[1].set_xlabel("Time [s]")
plt.show()

In [ ]:
from pydy.viz import Scene, Cylinder, Sphere, VisualizationFrame

cord_geom = Cylinder(length=L, radius=0.05, color='grey', name="cord")
particle_geom = Sphere(0.2, color="black", name="particle")

cord_vf = VisualizationFrame('cord vf', A, cord.masscenter, cord_geom)
particle_vf = VisualizationFrame('particle vf', N, particle.masscenter, particle_geom)

O = cord.masscenter.locatenew("O", -L/2 * cord.frame.y)
scene = Scene(M, O, cord_vf, particle_vf, system=sys)
scene.display_jupyter(axes_arrow_length=1)

In [ ]:
cord.loads

In [ ]:
wall.loads

In [ ]:
type(cord)